In [ ]:
import os 
import numpy as np
from collections import defaultdict

In [ ]:
#CALC THE MAX NUMBER OF HOURS FROM EXCEL SPREADSHEET
def dataCalc():
    data = np.genfromtxt(input_file, dtype=float, delimiter=",")
    # isolate Ansys STK Pro (stk_mission_level1) line
    stk_ms_lv1 = data[3]

    # calculates max time spent 
    x=0
    lv1max=0
    for key in stk_ms_lv1:
        if stk_ms_lv1[x] >= 1: # adds only whole numbers 
            lv1max += stk_ms_lv1[x]
        x+=1

    lv1max = str(lv1max)[:-2] # saves only the whole number 
    return lv1max

In [ ]:
#FUNCTION POPULATES DAT FILE DATA LINE 
def dataPrint(file):
    
    #SPACE PADDED BEFORE DATA
    def dat_formatEnd(dat,leng,spc,e=0):
        dat_filler = spc - leng
        while e < dat_filler:
            datLine.append(" ")
            e+=1
        datLine.append(dat)
 
    #SPACE PADDED AFTER DATA     
    def dat_formatBeg(dat,leng,spc,b=0):
        dat_filler = spc - leng
        datLine.append(dat)
        while b < dat_filler:
            datLine.append(" ")
            b+=1
   
    # lists assigned constants   
    P_S = "TETKS"
    OAC = "47"
    OBAN = "GV"
    PEC = "65807F"
    RCCC = "7JGJOA"
    EEIC = "63720"
    BPAC = "6606TS"
    input_file = file # declares input file 
    COM = input_file[9:21]
    
    # declare all variables 
    
    # item key: 0 = no information provided for data line || other = constant variable or calc value
    item_key = {'aid':0,'jon':0,'charge_date':0,'fc':0,'wbs':0 ,'rccc':RCCC,'ps_code': P_S,'eeic':EEIC,'trans_type':0,'quantity': str(dataCalc()),'amt':0,'fy':0,'reserved1':0,'local_use':0,'remarks':0,'contract_number':0,'reserved2':0,'fund_source':0,'reserved3':0,'job_type':0,'contractor_org_code':0,'work_phase_code':0,'facility_code':0,'main_effort':0,'sub_effort':0,'sow_annex':0,'oban_yr':0,'oban':OBAN,'oac':OAC,'pec':PEC,'rmb_ind':0,'location_code':0,'test_number':0,'contractor_support_card':0,'won':0,'equip_id_number':0,'location_area':0,'location_complex':0,'location_pad':0,'comments':COM,'bpac':BPAC,'ba':0,'sales_code':0} 
    item_col = {'aid':2,'jon':8,'charge_date':11,'fc':2,'wbs':4 ,'rccc':6,'ps_code':5,'eeic':5,'trans_type':1,'quantity':14,'amt':14,'fy':4,'reserved1':40,'local_use':6,'remarks':25,'contract_number':11,'reserved2':1,'fund_source':1,'reserved3':6,'job_type':2,'contractor_org_code':4,'work_phase_code':1,'facility_code':14,'main_effort':6,'sub_effort':3,'sow_annex':2,'oban_yr':1,'oban':2,'oac':2,'pec':6,'rmb_ind':1,'location_code':2,'test_number':5,'contractor_support_card':5,'won':8,'equip_id_number':10,'location_area':2,'location_complex':2,'location_pad':1,'comments':80,'bpac':6,'ba':2,'sales_code':3}

    item_val = list(item_key.values())
    item_spc = list(item_col.values())
    
    # variables to format QUANTITY section  
    quan = item_key.get("quantity")
    quan_str = str(quan)
    quan_len = len(quan_str)
    quan_spc = item_col.get("quantity")

    # variables to format COMMENTS section
    com = item_key.get("comments")
    com_len = len(com)
    com_spc = item_col.get("comments")
    
    # for loop prints out data line items
    datLine = [] # empty dictionary for data 
    x = 0
    for key in item_col: # iterates through item types
        i=0
        if item_val[x] == 0: # prints blank space for values that are not 
            while i < item_spc[x]:
                datLine.append(" ")
                i+=1
        elif key == "quantity": # formats quantity to populate back -> front
            dat_formatEnd(quan, quan_len, quan_spc)
        elif key == "comments": # formates comments to populate front -> back
            dat_formatBeg(com, com_len, com_spc)
        else: # prints constant variables / function returns 
            datLine.append(item_val[x])
        x+=1

    datLine_form = ''.join(datLine) # formats data line to print correctly 
    return datLine_form

In [ ]:
# for loop populates dictionary with every CSV file in immediate directory
file_dict = defaultdict(list)
i=1

#GETS FILES FROM IMMEDIATE DIRECTORY
for root, dirs, files in os.walk("."):
    for filename in files: # dictionary format = file1: filename.csv, file2: filename.csv
        if filename.endswith('.csv'):
            file_dict['file{}.'.format(str(i))].append(filename)
            i+=1
            
# for loop calls each file for manipulation
j=0
file_list = list(file_dict.values())
f = open("dataOutput.dat", "w") # RENAME output file HERE

#DRIVER FOR LOOP 
for key in file_dict: # iterates through each file in immediate directory
    input_file = ((str(file_list[j])[2:-2])) 
    if(input_file[0:4] != "Peak"):
            compName = input_file
    elif(input_file[0:4] == "Peak"): # assumes naming convention for billing data is like example
        datLine_form = dataPrint(input_file) 
        
        #WRITES TO .DAT FILE
        f.write(str(datLine_form))
        f.write("\n")

    if j<i:
        j+=1

f.close() # closes file write


# FOR TESTING ONLY 
r = open("dataOutput.dat", "r") # opens file read 
print(r.read())
r.close()  